In [1]:
import sys
sys.path.append('..')

In [2]:
from tofina.components import asset, preference
from tofina.theory import varianceMinimization
import tofina.utils as utils
from tofina.macros import portfolioOptimization, optionPricing
import pandas as pd
import numpy as np
import torch

minVarianceFilePath1 = "minVariancePortfolio1.csv"
covariance1 = torch.tensor([[5, 1], [1, 10]]).float() / 100
mean=0.03


portfolio_ = varianceMinimization.GenerateVarianceMinimizationPortfolio(
    mean, covariance1, torch.tensor([1.0, 1.0]), torch.tensor([1.0, 1.0])
)
optim = portfolioOptimization.optimizeStockPortfolioRiskAverse(
    portfolio_, minVarianceFilePath1
)



../tofina/utils.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  params[key] = torch.nn.Parameter(torch.tensor(val), requires_grad=False)
  9%|▉         | 88/1000 [00:00<00:03, 267.86it/s]


In [3]:
optim.optimizationResult

{'initial_Stock_Company0weight': 0.5,
 'initial_Stock_Company1weight': 0.5,
 'initial_loss': -0.019090916961431503,
 'final_Stock_Company0weight': 0.6950502991676331,
 'final_Stock_Company1weight': 0.30494970083236694,
 'final_loss': -0.020339922979474068,
 'converged': True}

In [1]:
import sys
import pandas as pd
import numpy as np
import yfinance as yf
from arch import arch_model
import datetime as dt
import torch
sys.path.append('..')

In [1]:
import sys
sys.path.append('..')
from tofina.components.backtest import Backtester
from tofina.extern.arch import forecastDecoratorGARCH
from tofina.extern.yfinance import loadHistoricalStockData
import datetime as dt
import numpy as np
from pathlib import Path


# TODO
# 1. Make forecaster compatible with Assets
# 2. Create realPath dictionary
# 3. Create backtest method
SPY = loadHistoricalStockData("SPY")
start_date = dt.datetime(2023, 1, 1)
end_date = dt.datetime(2024, 1, 1)
split_date = start_date - dt.timedelta(days=1)
timestamps = SPY.index[
    np.logical_and(SPY.index >= start_date, SPY.index < end_date)
]
horizon = 20
simulations = 1000
arch_kwargs = dict(vol="Garch", p=3, o=0, q=3, dist="t", mean="AR", lags=5)
forecaster = forecastDecoratorGARCH(
    SPY, split_date, horizon, simulations, **arch_kwargs
)
backtester = Backtester(timestamps=timestamps)
backtester.stockDataFromDataFrame(SPY, "SPY")
backtester.registerForecaster(forecaster, ["SPY"], allowShorting=False)
dir_path = Path("./tests/results/SPY_GARCH")
if dir_path.exists():
    dir_path.rmdir()
dir_path.mkdir(parents=True, exist_ok=False)
backtester.optimizeStrategy(dir_path)

[*********************100%%**********************]  1 of 1 completed


Iteration:      5,   Func. Count:     94,   Neg. LLF: 30356.489405585766
Iteration:     10,   Func. Count:    181,   Neg. LLF: 15244.176184581413
Iteration:     15,   Func. Count:    263,   Neg. LLF: 10144.559858087046
Iteration:     20,   Func. Count:    344,   Neg. LLF: 10075.081632452344
Iteration:     25,   Func. Count:    420,   Neg. LLF: 10073.148971941075
Optimization terminated successfully    (Exit mode 0)
            Current function value: 10073.14896934825
            Iterations: 27
            Function evaluations: 449
            Gradient evaluations: 27


/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


TypeError: forecast() got multiple values for argument 'date'

In [2]:
%debug

> /Users/andriylevitskyy/Desktop/tofina/tofina/components/asset.py(40)simulate()
     38 
     39     def simulate(self) -> torch.Tensor:
---> 40         return self.processFn(self.processLength, self.monteCarloTrials, self.params)
     41 
     42 

> /Users/andriylevitskyy/Desktop/tofina/tofina/components/asset.py(31)__init__()
     29         self.monteCarloTrials = monteCarloTrials
     30         self.params = utils.convertKwargsToTorchParameters(kwargs)
---> 31         self.monteCarloSimulation = self.simulate()
     32         if len(self.monteCarloSimulation.shape) == 3:
     33             assert type(name) is list

> /Users/andriylevitskyy/Desktop/tofina/tofina/components/portfolio.py(17)addAsset()
     15         self, name: Union[str, List[str]], processFn: asset.processFnType, **kwargs
     16     ) -> None:
---> 17         asset_ = asset.Asset(
     18             name, processFn, self.processLength, self.monteCarloTrials, **kwargs
     19         )

> /Users/andriylevits